## Day 46 Lecture 2 Assignment

In this assignment, we will calculate a similarity matrix using responses to a survey about student life at a university. We will perform clustering on this dataset later on.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [2]:
# answer goes here
raw_data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')
raw_data

,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,Q3-Interested in Volunteering,Q3-Interested in Others,Q4-Passionate about Animal welfare,Q4-Passionate about Arts/Culture/Heritage,Q4-Passionate about Children/Youth,Q4-Passionate about Community building,"Q4-Passionate about Diversity & Inclusion (e.g. special needs, migrant worker, interfaith and intercultural understanding)",Q4-Passionate about Environmental sustainability,Q4-Passionate about Families,Q4-Passionate about Health/Well-being (e.g mental health),Q4-Passionate about Seniors,Q4-Passionate about Poverty reduction,Q4-Passionate about Education,Q4-Passionate about None of the above,Q4-Passionate about Others,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,1,0,0,0,1,1,1,1,0,0,1,0,1,1,0,0,0,1,1,1,1,1,1,0,0,0,1,1,0,0,1,0,0,0,0,2954
2954,1,0,0,0,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,2955
2955,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2956
2956,0,1,0,1,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,2957


For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [3]:
df_q5 = raw_data.iloc[:, -11:-1]
df_q5

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
2953,0,1,1,0,0,1,0,0,0,0
2954,0,1,0,0,0,0,0,0,0,0
2955,0,1,0,0,0,0,0,0,0,0
2956,0,1,0,0,0,0,1,1,0,0


We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

The pdist function is a simple way to accomplish this goal, and the squareform function will transform the output of the pdist function into a tabular format that can be convert to a dataframe more easily.

In [63]:
hamming = pdist(df_q5, metric='hamming')
hamming = pd.DataFrame(squareform(hamming), index=df_q5.index, columns=df_q5.index)
hamming

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2918,2919,2920,2921,2922,2923,2924,2925,2926,2927,2928,2929,2930,2931,2932,2933,2934,2935,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946,2947,2948,2949,2950,2951,2952,2953,2954,2955,2956,2957
0,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
1,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2,0.1,0.1,0.0,0.7,0.4,0.2,0.1,0.2,0.5,0.3,0.2,0.2,0.4,0.5,0.3,0.2,0.2,0.3,0.3,0.4,0.2,0.3,0.2,0.1,0.2,0.3,0.2,0.1,0.4,0.0,0.2,0.2,0.4,0.2,0.4,0.0,0.2,0.2,0.4,0.1,...,0.3,0.1,0.1,0.0,0.1,0.3,0.0,0.1,0.2,0.1,0.3,0.1,0.2,0.3,0.2,0.1,0.1,0.0,0.1,0.0,0.2,0.2,0.1,0.3,0.1,0.2,0.3,0.0,0.3,0.1,0.2,0.4,0.3,0.4,0.4,0.3,0.1,0.1,0.1,0.1
3,0.6,0.6,0.7,0.0,0.5,0.5,0.8,0.7,0.2,0.6,0.7,0.5,0.5,0.4,0.6,0.7,0.7,0.6,0.6,0.7,0.7,0.6,0.7,0.6,0.5,0.6,0.7,0.8,0.5,0.7,0.7,0.7,0.7,0.5,0.5,0.7,0.7,0.7,0.7,0.6,...,0.6,0.6,0.8,0.7,0.6,0.4,0.7,0.6,0.7,0.6,0.4,0.6,0.5,0.6,0.5,0.8,0.6,0.7,0.6,0.7,0.5,0.5,0.6,0.6,0.6,0.7,0.6,0.7,0.4,0.6,0.7,0.5,0.4,0.5,0.3,0.6,0.6,0.6,0.6,0.8
4,0.3,0.3,0.4,0.5,0.0,0.2,0.3,0.6,0.3,0.3,0.4,0.4,0.4,0.5,0.5,0.6,0.4,0.5,0.3,0.4,0.2,0.3,0.4,0.3,0.4,0.3,0.2,0.3,0.4,0.4,0.4,0.2,0.4,0.4,0.4,0.4,0.2,0.4,0.4,0.3,...,0.3,0.3,0.3,0.4,0.3,0.3,0.4,0.5,0.2,0.3,0.3,0.3,0.2,0.3,0.4,0.5,0.5,0.4,0.3,0.4,0.4,0.4,0.3,0.5,0.3,0.2,0.3,0.4,0.3,0.5,0.2,0.2,0.3,0.4,0.2,0.3,0.3,0.3,0.5,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,0.2,0.2,0.3,0.6,0.3,0.3,0.2,0.5,0.4,0.4,0.3,0.5,0.3,0.4,0.4,0.5,0.1,0.6,0.2,0.3,0.3,0.2,0.3,0.2,0.5,0.0,0.3,0.2,0.1,0.3,0.3,0.3,0.5,0.3,0.5,0.3,0.1,0.3,0.3,0.2,...,0.2,0.4,0.2,0.3,0.2,0.6,0.3,0.4,0.1,0.2,0.6,0.2,0.3,0.4,0.3,0.4,0.2,0.3,0.2,0.3,0.5,0.1,0.2,0.4,0.2,0.1,0.2,0.3,0.4,0.4,0.1,0.1,0.4,0.3,0.5,0.0,0.2,0.2,0.4,0.2
2954,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2955,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2956,0.2,0.2,0.1,0.6,0.5,0.3,0.2,0.3,0.6,0.4,0.3,0.3,0.5,0.4,0.4,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.2,0.3,0.4,0.3,0.2,0.5,0.1,0.1,0.3,0.5,0.3,0.5,0.1,0.3,0.3,0.5,0.2,...,0.4,0.2,0.2,0.1,0.2,0.4,0.1,0.0,0.3,0.2,0.4,0.2,0.3,0.4,0.1,0.2,0.2,0.1,0.2,0.1,0.3,0.3,0.2,0.4,0.2,0.3,0.4,0.1,0.2,0.0,0.3,0.5,0.4,0.5,0.5,0.4,0.2,0.2,0.0,0.2


Print the answers to the following three questions:

1. How many pairs of students are there in our dataset? (Remember to ignore a student pairing with themselves, and avoid double-counting - '3-5' is the same as '5-3'.) 
2. Of these pairs, how many gave identical answers? 
3. How many pairs had no answers in common?

Tip: it may help to first convert the NxN similarity matrix into a table with three columns: response_id_1, response_id_2, and the similarity between them. The new table should have NxN rows and 3 columns.

In [64]:
# answer goes here
# answer goes here
sim = pdist(hamming, metric='matching')
sim = pd.DataFrame(squareform(sim), index=hamming.index, columns=hamming.columns)

sim['id'] = sim.index
sim_melted = pd.melt(sim,id_vars=['id'], value_vars=sim.columns[sim.columns != 'id'])
sim_melted.columns = ['response_id_1', 'response_id_2', 'dissimilarity']

sim_melted_condensed = sim_melted[sim_melted.response_id_1 < sim_melted.response_id_2]

In [65]:
sim_melted_condensed

,response_id_1,response_id_2,dissimilarity
2958,0,1,0.000000
5916,0,2,1.000000
5917,1,2,1.000000
8874,0,3,0.908384
8875,1,3,0.908384
...,...,...,...
8749758,2952,2957,1.000000
8749759,2953,2957,0.446247
8749760,2954,2957,0.556457
8749761,2955,2957,0.556457


In [74]:
print(f'There are {sim_melted_condensed.shape[0]} pairs of students in the dataset')

There are 4373403 pairs of students in the dataset


In [75]:
same = (sim_melted_condensed['dissimilarity'] == 0).value_counts()
same

False    4236671
True      136732
Name: dissimilarity, dtype: int64

In [76]:
print(f'There are {same[1]} pairs with identical answers')

There are 136732 pairs with identical answers


In [86]:
different = (sim_melted_condensed['dissimilarity'] == 1).value_counts()
different

False    2188771
True     2184632
Name: dissimilarity, dtype: int64

In [78]:
print(f'There are {different[1]} pairs with completely different answers')

There are 2184632 pairs with completely different answers


For each student, calculate their average similarity with all other students. Then print the following:

1. On average, which student (i.e. which response_id) had the most similar responses to the survey population as a whole? What was the average similarity for this student, and what were their answers?
2. Repeat part 1, but for the student with the least similar responses to the survey population as a whole.
3. What do we notice about these students? What broader inferences can we make about the student population as a whole?

In [87]:
# answer goes here

#1

print("-----------Average Similarity for each student-----------")
result = sim_melted_condensed.groupby(by=['response_id_1']).mean().sort_values(by='dissimilarity', ascending=True)
print(result)
print("----------Most Similar student-----------")
print(result.iloc[[0]])
print("----------Responses for Most Similar Student-----------")
print(df_q5.iloc[2954])





-----------Average Similarity for each student-----------
               dissimilarity
response_id_1               
2954                0.369957
2956                0.382353
2953                0.542934
2955                0.554936
2940                0.649107
...                      ...
2041                0.930543
1796                0.931129
2416                0.933913
2899                0.939101
2952                1.000000

[2957 rows x 1 columns]
----------Most Similar student-----------
               dissimilarity
response_id_1               
2954                0.369957
----------Responses for Most Similar Student-----------
Q5-Stressed about Adjustment issues                                                    0
Q5-Stressed about Academic issues                                                      1
Q5-Stressed about Financial issues                                                     0
Q5-Stressed about Family issues                                                        0

In [88]:
# 2

print("----------Most Dissimilar student-----------")
print(result.iloc[[-1]])
print("----------Responses for Most Similar Student-----------")
print(df_q5.iloc[2899])


----------Most Dissimilar student-----------
               dissimilarity
response_id_1               
2952                     1.0
----------Responses for Most Similar Student-----------
Q5-Stressed about Adjustment issues                                                    1
Q5-Stressed about Academic issues                                                      1
Q5-Stressed about Financial issues                                                     1
Q5-Stressed about Family issues                                                        1
Q5-Stressed about Friendships                                                          1
Q5-Stressed about Romantic relationships                                               0
Q5-Stressed about  Health related issues                                               1
Q5-Stressed about Career related issues                                                1
Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.    1
Q5-Stressed